# Robyn Budget Allocator Demo

This notebook demonstrates how to use the Python implementation of Robyn's budget allocator.
It shows how to:
1. Load and prepare data
2. Configure the allocator
3. Run optimization scenarios
4. Analyze and visualize results

## Step 1: Load Exported R Data

In [1]:
import sys
import pandas as pd
import numpy as np
from typing import Dict, Any

# Add Robyn to path
sys.path.append("/Users/yijuilee/robynpy_release_reviews/Robyn/python/src")

# Import necessary Robyn classes
from robyn.data.entities.mmmdata import MMMData
from robyn.modeling.entities.modeloutputs import ModelOutputs
from robyn.data.entities.hyperparameters import Hyperparameters
from robyn.allocator.entities.enums import OptimizationScenario, ConstrMode
from robyn.allocator.budget_allocator import BudgetAllocator
from robyn.modeling.pareto.pareto_optimizer import ParetoResult
from robyn.allocator.entities.allocation_config import AllocationConfig
from robyn.allocator.entities.allocation_constraints import AllocationConstraints
from utils.data_mapper import load_data_from_json, import_input_collect, import_output_collect, import_output_models

/Users/yijuilee/robynpy_release_reviews/robynvenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'Any' is not defined

In [ ]:
# Load data from JSON exported from R
raw_input_collect = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_InputCollect.json"
)
raw_output_collect = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputCollect.json"
)
raw_output_models = load_data_from_json(
    "/Users/yijuilee/robynpy_release_reviews/Robyn/python/src/tutorials/data/Allocator_OutputModels.json"
)

# Convert R data to Python objects
r_input_collect = import_input_collect(raw_input_collect)
r_output_collect = import_output_collect(raw_output_collect)
python_model_outputs = import_output_models(raw_output_models)

# Extract individual components
mmm_data = r_input_collect["mmm_data"]
model_outputs = python_model_outputs
hyperparameters = r_input_collect["hyperparameters"]

# Print data summary
print(f"Data loaded successfully:")
print(
    f"- Data timeframe: {mmm_data.data[mmm_data.mmmdata_spec.date_var].min()} to {mmm_data.data[mmm_data.mmmdata_spec.date_var].max()}"
)
print(f"- Number of paid media channels: {len(mmm_data.mmmdata_spec.paid_media_spends)}")
print(f"- Channels: {mmm_data.mmmdata_spec.paid_media_spends}")

## Step 2: Set up Budget Allocator

Initialize the budget allocator with the selected model and data.

In [ ]:
# # Print some debugging information
# print(f"Model ID: {select_model}")
# print("\nAvailable models in result_hyp_param:")
# print(r_output_collect["pareto_result"].result_hyp_param["solID"].unique())
# print("\nColumns in result_hyp_param:")
# print(r_output_collect["pareto_result"].result_hyp_param.columns)

In [ ]:
# Select model ID from available solutions
select_model = "1_208_4"  # This should match one of your model IDs

# Initialize budget allocator
allocator = BudgetAllocator(
    mmm_data=mmm_data,
    model_outputs=model_outputs,
    pareto_result=r_output_collect["pareto_result"],  # Get ParetoResult from import_output_collect()
    select_model=select_model,
)

## Step 3: Configure Allocation Settings

Set up constraints and parameters for optimization.

In [ ]:
# Create channel constraints
channel_constraints = AllocationConstraints(
    channel_constr_low={channel: 0.7 for channel in mmm_data.mmmdata_spec.paid_media_spends},
    channel_constr_up={channel: 1.2 for channel in mmm_data.mmmdata_spec.paid_media_spends},
    channel_constr_multiplier=3.0,
)

# Create allocation configuration for max response scenario
max_response_config = AllocationConfig(
    scenario=OptimizationScenario.MAX_RESPONSE,
    constraints=channel_constraints,
    date_range="last",  # Use last period as initial
    total_budget=None,  # Use historical budget
    maxeval=100000,
    optim_algo="SLSQP_AUGLAG",
    constr_mode=ConstrMode.EQUALITY,
    plots=True,
)

## Step 4: Run Optimization Scenarios

### Scenario 1: Maximize Response
Find optimal allocation to maximize response while keeping total budget constant.

In [ ]:
# Run max response optimization
max_response_result = allocator.allocate(max_response_config)

# Print optimization results
print("\nMax Response Optimization Results:")
print(max_response_result.summary)

# Display optimal allocations
print("\nOptimal Channel Allocations:")
print(max_response_result.optimal_allocations)

### Scenario 2: Target Efficiency
Optimize allocation based on target ROI/CPA.

In [ ]:
# Create configuration for target efficiency scenario
target_efficiency_config = AllocationConfig(
    scenario=OptimizationScenario.TARGET_EFFICIENCY,
    constraints=channel_constraints,
    date_range="last",
    target_value=None,  # Will use default 80% of initial ROAS or 120% of initial CPA
    maxeval=100000,
    optim_algo="SLSQP_AUGLAG",
    constr_mode=ConstrMode.EQUALITY,
    plots=True,
)

# Run target efficiency optimization
target_efficiency_result = allocator.allocate(target_efficiency_config)

# Print optimization results
print("\nTarget Efficiency Optimization Results:")
print(target_efficiency_result.summary)

# Display optimal allocations
print("\nOptimal Channel Allocations:")
print(target_efficiency_result.optimal_allocations)

## Step 5: Analyze Results

Compare and analyze the results from different scenarios.

In [ ]:
def compare_scenarios(max_response_result, target_efficiency_result):
    """Compare results from different optimization scenarios."""

    comparison = pd.DataFrame(
        {
            "Max Response": max_response_result.optimal_allocations["optimal_spend"],
            "Target Efficiency": target_efficiency_result.optimal_allocations["optimal_spend"],
        }
    )

    comparison["Difference"] = comparison["Max Response"] - comparison["Target Efficiency"]
    comparison["Difference %"] = (comparison["Difference"] / comparison["Max Response"]) * 100

    return comparison


# Generate comparison
scenario_comparison = compare_scenarios(max_response_result, target_efficiency_result)
print("\nScenario Comparison:")
print(scenario_comparison)

## Step 6: Visualize Results

Plot the optimization results and response curves.

In [ ]:
# Plot functions would be implemented here to visualize:
# - Response curves for each channel
# - Current vs optimized allocation comparison
# - ROI/CPA comparison
# - Total response comparison